In [1]:
import os
import sys
import tensorflow as tf

sys.path.append("/home/student/Documents/MDS22/Aneurysm_Detection_dev")
from training.network_training import create_compiled_unet

2024-04-25 10:51:22.339962: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 10:51:22.339995: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 10:51:22.341017: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 10:51:22.346822: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 10:51:23.005708: W tensorflow/comp

In [2]:
# define input and create U-Net model
inputs = tf.keras.Input(shape=(64, 8, 8, 1), name='TOF_patch')
# create UNET and compile. There shouldn't be a need to compile since we're doing inference, but this solves a TF bug of the predict method, so we must compile the model
unet = create_compiled_unet(inputs, 0.0001, 0.5, [9, 26, 61, 74])
# LOAD weights saved from a trained model somewhere else
unet.load_weights(os.path.join("/home/student/Documents/MDS22/Aneurysm_Detection_dev/fixed_pickles/Train_Outputs_Aug_16_2021_chuv_weak_plus_voxelwise_pretrain_adam_github/fold1/saved_models", "my_checkpoint")).expect_partial()

2024-04-25 10:51:26.458092: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15081 MB memory:  -> device: 0, name: Quadro P5000, pci bus id: 0000:91:00.0, compute capability: 6.1
/home/student/miniconda3/envs/fyp2/lib/python3.9/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



In [7]:
unet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 TOF_patch (InputLayer)      [(None, 64, 8, 8, 1)]        0         []                            
                                                                                                  
 conv3d_18 (Conv3D)          (None, 64, 8, 8, 9)          252       ['TOF_patch[0][0]']           
                                                                                                  
 conv3d_19 (Conv3D)          (None, 64, 8, 8, 9)          2196      ['conv3d_18[0][0]']           
                                                                                                  
 batch_normalization_7 (Bat  (None, 64, 8, 8, 9)          36        ['conv3d_19[0][0]']           
 chNormalization)                                                                           

In [3]:
import torch
import tf2onnx
import onnx
from onnx2pytorch import ConvertModel

# Convert the model to ONNX format
onnx_model, _ = tf2onnx.convert.from_keras(unet)

# Load ONNX model
# onnx_model = onnx.load_model("tf_model.onnx")

# Convert ONNX model to PyTorch
pytorch_model = ConvertModel(onnx_model)

2024-04-25 10:51:34.314485: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-04-25 10:51:34.314595: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-04-25 10:51:35.990001: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15081 MB memory:  -> device: 0, name: Quadro P5000, pci bus id: 0000:91:00.0, compute capability: 6.1
2024-04-25 10:51:36.228482: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15081 MB memory:  -> device: 0, name: Quadro P5000, pci bus id: 0000:91:00.0, compute capability: 6.1
2024-04-25 10:51:36.260622: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-04-25 10:51:36.260750: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new sess

In [4]:
pytorch_model.network

AttributeError: 'ConvertModel' object has no attribute 'network'

In [16]:
from medcam import medcam

medcam_model = medcam.inject(pytorch_model, output_dir="attention_maps", save_maps=True)